In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np

# configurando as cores
amarelo = '#FFD744'
vermelho = '#A80000'
azul = '#183154'
branco = "#FFFFFF"

# importando dados do state of data de 2022 (mais atual) e eliminando os duplicados
df = pd.read_csv('data/State_of_data_2022.csv')
df = df.drop_duplicates()

# Renomeando as Colunas para melhor uso

df.rename(columns={ # colunas sobre a identidade dos colaboradores de 2022
    "('P0', 'id')" : "id",
    "('P2_a ', 'Qual sua situação atual de trabalho?')" : "situacao_atual",
    "('P2_c ', 'Numero de Funcionarios')" : 'numero_de_funcionarios',
    "('P2_d ', 'Gestor?')" : "gestor",
    "('P2_h ', 'Faixa salarial')": "faixa_salario",
    "('P2_k ', 'Você está satisfeito na sua empresa atual?')" : "satisfeito",

    # Colunas sobre modelo de trabalho remoto de colaboradores de 2022
    "('P2_p ', 'Atualmente qual a sua forma de trabalho?')" : 'modelo_de_trabalho',
    "('P2_r ', 'Caso sua empresa decida pelo modelo 100% presencial qual será sua atitude?')" : "volta_ao_presencial",


    # Coluna sobre querer mudar de emprego
    "('P2_n ', 'Você pretende mudar de emprego nos próximos 6 meses?')" : "mudar_de_emprego",

    # Colunas sobre os motivos de insatisfação de colaboradores de 2022
    "('P2_l_1 ', 'Falta de oportunidade de crescimento no emprego atual')" : "oportunidade",
    "('P2_l_2 ', 'Salário atual não corresponde ao mercado')" : "salario_mercado",
    "('P2_l_3 ', 'Não tenho uma boa relação com meu líder/gestor')" : "lider",
    "('P2_l_4 ', 'Gostaria de trabalhar em em outra área de atuação')" : "area",
    "('P2_l_5 ', 'Gostaria de receber mais benefícios')" : "beneficios",
    "('P2_l_6 ', 'O clima de trabalho/ambiente não é bom')": "clima",
    "('P2_l_7 ', 'Falta de maturidade analítica na empresa')" : 'maturidade',
    "('P3_c_1 ', 'Pensar na visão de longo prazo de dados da empresa e fortalecimento da cultura analítica da companhia.')" : "visao",

    # tamnho da equipe
    "('P3_a ', 'Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?')" : "tamanho_equipe",

    # Colunas sobre desafios e funções dos gestores de 2022
    "('P3_c_2 ', 'Organização de treinamentos e iniciativas com o objetivo de aumentar a maturidade analítica das áreas de negócios.')" : "treinamento",
    "('P3_c_3 ', 'Atração, seleção e contratação de talentos para o time de dados.')" : "rh",
    "('P3_c_4 ', 'Decisão sobre contratação de ferramentas e tecnologias relacionadas a dados.')" : "ferramentas",
    "('P3_c_5 ', 'Sou gestor da equipe responsável pela engenharia de dados e por manter o Data Lake da empresa como fonte única dos dados, garantindo a qualidade e confiabilidade da informação.')" : "data_lake",
    "('P3_c_6 ', 'Sou gestor da equipe responsável pela entrega de dados, estudos, relatórios e dashboards para as áreas de negócio da empresa.')" : "gestor_equipe",
    "('P3_c_7 ', 'Sou gestor da equipe responsável por iniciativas e projetos envolvendo Inteligência Artificial e Machine Learning.')" : "gestor_ia",
    "('P3_c_8 ', 'Apesar de ser gestor ainda atuo na parte técnica, construindo soluções/análises/modelos etc.')": "parte_tecnica",
    "('P3_c_9 ', 'Gestão de projetos de dados, cuidando das etapas, equipes envolvidas, atingimento dos objetivos etc.')": "gestor_etapa",
    "('P3_c_10 ', 'Gestão de produtos de dados, cuidando da visão dos produtos, backlog, feedback de usuários etc.')": "gestor_produto",
    "('P3_c_11 ', 'Gestão de pessoas, apoio no desenvolvimento das pessoas, evolução de carreira')": "desenvolvimento",
    "('P3_d ', 'Quais são os 3 maiores desafios que você tem como gestor no atual momento?')" : "desafios_gestor"
    },inplace=True)


def formatar_df(df, coluna):
    """
    Formata um DataFrame contendo informações de satisfação de acordo com a coluna especificada.

    Esta função realiza o seguinte processamento:
    1. Remove linhas com valores ausentes na coluna 'satisfeito'.
    2. Gera uma lista de valores únicos na coluna especificada.
    3. Calcula a contagem de satisfeitos e insatisfeitos para cada valor na coluna especificada.
    4. Calcula as porcentagens de satisfeitos e insatisfeitos em relação ao total para cada valor.
    
    Args:
        df (pandas.DataFrame): O DataFrame de entrada contendo os dados de satisfação.
        coluna (str): O nome da coluna com os valores para os quais a análise de satisfação será realizada.

    Returns:
        pandas.DataFrame: Um novo DataFrame formatado com as colunas ['coluna', 'satisfeitos', 'insatisfeitos',
                         'total', 'por_centagem_satisfeitos', 'por_centagem_insatisfeitos'].
    """
    df_limpo = df.dropna(subset=['satisfeito'])
    lista_valores = list(df_limpo.dropna(subset=[coluna]).drop_duplicates(subset=[coluna])[coluna].values)
    lista_satisfacao = []
    satisfeitos = df_limpo.query('satisfeito == True').groupby([coluna])['id'].count()
    insatisfeitos = df_limpo.query('satisfeito == False').groupby([coluna])['id'].count()
    
    for valor in lista_valores:
        total = satisfeitos.get(valor, 0) + insatisfeitos.get(valor, 0)
        lista_satisfacao.append([valor, satisfeitos.get(valor, 0), insatisfeitos.get(valor, 0), total])
    
    df_final = pd.DataFrame(lista_satisfacao, columns=[coluna, 'satisfeitos', 'insatisfeitos', 'total'])
    df_final['por_centagem_satisfeitos'] = (df_final['satisfeitos'] * 100) / df_final['total']
    df_final['por_centagem_insatisfeitos'] = (df_final['insatisfeitos'] * 100) / df_final['total']
    
    return df_final

In [16]:
# selecionando as colunas que serão usadas
df_gestor = df[["id", "gestor","visao", "treinamento", "rh", "ferramentas", "data_lake", "gestor_equipe", "gestor_ia", "parte_tecnica", "gestor_etapa",  "gestor_produto", "desenvolvimento"]]

# Filtrando por apenas gestores e excluindo os arquivos nulos
df_gestor = df_gestor[df_gestor['gestor'] == True]
df_gestor = df_gestor.dropna()

# Configuração das cores e das hachuras
colors = [branco] * 11
colors[8] = vermelho
hachura = ['/'] * 11
hachura[8] = ''

# Criando Gráfico de Barras
fig = go.Figure([go.Bar(

    # informações do eixo y
    y=['Gestor da equipe responsável pela engenharia de dados ', 'Gestor da equipe de projetos envolvendo IA ', 'Gestão de produtos de dados ',
        'Ainda atuo na parte Técnica ', 'Decisão sobre contratação de Ferramentas ', 'Organização de Treinamentos ',
        'Gestão de projetos de Dados ', 'Gestor da equipe responsável pela entrega de dados e relatórios ',
        '<b>Atração, Seleção e Contratação de Talentos</b> ',
        'Apoio no Desenvolvimento das Pessoas ', 'Pensar na visão de longo prazo de dados da empresa '],

    # informações do eixo x
    x=[df_gestor['data_lake'].sum(), df_gestor['gestor_ia'].sum(), df_gestor['gestor_produto'].sum(),
        df_gestor['parte_tecnica'].sum(), df_gestor['ferramentas'].sum(), df_gestor['treinamento'].sum(),
        df_gestor['gestor_etapa'].sum(), df_gestor['gestor_equipe'].sum(), df_gestor['rh'].sum(),
        df_gestor['desenvolvimento'].sum(), df_gestor['visao'].sum()],

    # hachura, cores, borda e orientação do gráfico
    # marker_pattern_shape=hachura,
    marker_color=colors,
    marker_line=dict(width=1.8, color='black'),
    orientation='h'
    )])

# informações de tamanho e fonte
fig.update_layout(
    title={
    'text': "<b>Principais Funções dos Gestores em 2022</b>",
    'font' : dict(size=26, color='black', family='Nunito'),
    'y':0.95,
    'x':0.5},
    height=400, width=830,
    font_family='Nunito',
    font_color='black',
    font_size=16,
    plot_bgcolor='white',
    margin = {'pad': 5}
    
)

fig.show(config = {'displayModeBar': False})